In [39]:
import pandas as pd
import numpy as np
import CustomClassifier
reload(CustomClassifier)
import CustomFeatureTransformer
reload(CustomFeatureTransformer)
import random
from sklearn.cross_validation import train_test_split
from CustomFeatureTransformer import AreaRugTransformer
from CustomClassifier import AreaRugClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import LinearSVC
import re, time
from scipy.sparse import hstack
import os

df = pd.read_csv("/Users/a0m02fp/area_rug_tagged_data.csv")

df[u'product_name'].apply(str)
df[u'product_short_description'].apply(str)
df[u'product_long_description'].apply(str)
df[u'Manual curation'].apply(str)

# df = df.loc[df[u'Manual curation'] != 'None']

df_title_data = lambda dataF: list(dataF[u'product_name'])
df_desc_data = lambda dataF: list(dataF[u'product_short_description'] + " " + dataF[u'product_long_description'])

In [35]:
msk = np.random.rand(len(df)) < 0.8

train_df, test_df = df[msk], df[~msk]

train_sents_title, test_sents_title = map(lambda x:str(x), df_title_data(train_df)), map(lambda x:str(x), df_title_data(test_df))
train_sents_desc, test_sents_desc = map(lambda x:str(x), df_desc_data(train_df)), map(lambda x:str(x), df_desc_data(test_df))

train_labels, test_labels = list(train_df[u'Manual curation']), list(test_df[u'Manual curation'])

In [40]:
start = time.time()
transformer_title, transformer_desc = AreaRugTransformer(num_features=30, max_ngram=2), AreaRugTransformer(num_features=30, max_ngram=2)

transformer_title.fit(train_sents_title, train_labels)
transformer_desc.fit(train_sents_desc, train_labels)

train_title_data, train_desc_data = transformer_title.transform(train_sents_title), transformer_desc.transform(train_sents_desc)

train_data = hstack((train_title_data, train_desc_data))
print time.time()-start

11.0485470295


In [41]:
start = time.time()
base_estimator = LogisticRegression(penalty='l1')

model = AreaRugClassifier(base_estimator, train_title_data.shape[1], predict_threshold=0.5)
model.fit(train_data, train_labels)

test_title_data, test_desc_data = transformer_title.transform(test_sents_title), transformer_desc.transform(test_sents_desc)

test_data = hstack((test_title_data, test_desc_data))

print model.score(test_data, test_labels)
print time.time()-start

             precision    recall  f1-score   support

Hand-Knotted Rugs       0.99      0.88      0.93       182
Hand-Tufted Rugs       0.97      0.83      0.89       299
       None       0.95      0.99      0.97      1494

avg / total       0.96      0.96      0.96      1975

1.40019798279


In [42]:
%%timeit
test_title_data, test_desc_data = transformer_title.transform(test_sents_title[0:1]), transformer_desc.transform(test_sents_desc[0:1])
test_data = hstack((test_title_data, test_desc_data))
model.predict(test_data)

100 loops, best of 3: 2.33 ms per loop


In [13]:
test_sents_desc[0:1]

["10x12 Mashad Persian Area Rug This rug comes with free shipping and free 30 days return for full refund with no question asked. At Rugsource we offer 100% Satisfaction guaranteed at the wholesale price so everyone can have a beautiful rug in their house to bring warmth and joy by adding a handmade rug.<br><ul><li>Rug Age : 40-50 Years Old</li><li>Condition : Very Good ( Low Pile )</li><li>Rug Style : Mashad</li><li>Primary Color : Red &amp; Burgundies</li><li>Weave Type : Hand Knotted</li><li>Rug Size : 12' 2'' X 9' 9''</li><li>Rug Shape : Rectangle</li></ul>"]

In [5]:
model_folder = '/log/models/color_category'

class_names_file = os.path.join(model_folder, 'class_names.pkl')
title_transform_file = os.path.join(model_folder, 'titles_transform.pkl')
description_transform_file = os.path.join(model_folder, 'description_transform.pkl')
model_file = os.path.join(model_folder, 'model.pkl')

joblib.dump(model.class_labels, class_names_file)
joblib.dump(model, model_file)
joblib.dump(transformer_title, title_transform_file)
joblib.dump(transformer_desc, description_transform_file)

['/log/models/color_category/description_transform.pkl']

In [ ]:
pred_labels = model.mlb.inverse_transform(model.predict(test_data))
pred_labels = map(lambda x:'__'.join(x), pred_labels)

test_df['Predicted Label'] = pred_labels

test_df.to_csv('predictions.csv')

In [61]:
pred_labels = model.predict(test_data)
test_df['Inhouse_Model_Predicted'] = pred_labels
test_df.to_csv("/Users/a0m02fp/Predictions_Area_Rug.csv", encoding='utf-8', sep=",")

In [59]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kf.split(df):
    train_df, test_df = df.iloc[train_index], df.iloc[test_index]
    
    train_sents_title, test_sents_title = map(lambda x:str(x), df_title_data(train_df)), map(lambda x:str(x), df_title_data(test_df))
    train_sents_desc, test_sents_desc = map(lambda x:str(x), df_desc_data(train_df)), map(lambda x:str(x), df_desc_data(test_df))

    train_labels, test_labels = list(train_df[u'Manual curation']), list(test_df[u'Manual curation'])

    transformer_title, transformer_desc = AreaRugTransformer(), AreaRugTransformer()

    transformer_title.fit(train_sents_title, train_labels, num_features=20)
    transformer_desc.fit(train_sents_desc, train_labels, num_features=50)

    train_title_data, train_desc_data = transformer_title.transform(train_sents_title), transformer_desc.transform(train_sents_desc)

    train_data = hstack((train_title_data, train_desc_data))

    base_estimator = LogisticRegression(penalty='l1')

    model = AreaRugClassifier(base_estimator, train_title_data.shape[1])
    model.fit(train_data, train_labels)

    test_title_data, test_desc_data = transformer_title.transform(test_sents_title), transformer_desc.transform(test_sents_desc)

    test_data = hstack((test_title_data, test_desc_data))

    print model.score(test_data, test_labels)

ImportError: No module named model_selection

In [56]:
import numpy as np
a = np.array([[2,3,5],[5,1,9],[7,8,0]])
b = np.array([[6,9,1],[3,2,5],[1,6,3]])
print a
print b
np.maximum.reduce([a,b])

[[2 3 5]
 [5 1 9]
 [7 8 0]]
[[6 9 1]
 [3 2 5]
 [1 6 3]]


array([[6, 9, 5],
       [5, 2, 9],
       [7, 8, 3]])

In [6]:
from nltk.corpus import stopwords

In [7]:
stopwords.words('english')

[u'i',
 u'me',
 u'my',
 u'myself',
 u'we',
 u'our',
 u'ours',
 u'ourselves',
 u'you',
 u"you're",
 u"you've",
 u"you'll",
 u"you'd",
 u'your',
 u'yours',
 u'yourself',
 u'yourselves',
 u'he',
 u'him',
 u'his',
 u'himself',
 u'she',
 u"she's",
 u'her',
 u'hers',
 u'herself',
 u'it',
 u"it's",
 u'its',
 u'itself',
 u'they',
 u'them',
 u'their',
 u'theirs',
 u'themselves',
 u'what',
 u'which',
 u'who',
 u'whom',
 u'this',
 u'that',
 u"that'll",
 u'these',
 u'those',
 u'am',
 u'is',
 u'are',
 u'was',
 u'were',
 u'be',
 u'been',
 u'being',
 u'have',
 u'has',
 u'had',
 u'having',
 u'do',
 u'does',
 u'did',
 u'doing',
 u'a',
 u'an',
 u'the',
 u'and',
 u'but',
 u'if',
 u'or',
 u'because',
 u'as',
 u'until',
 u'while',
 u'of',
 u'at',
 u'by',
 u'for',
 u'with',
 u'about',
 u'against',
 u'between',
 u'into',
 u'through',
 u'during',
 u'before',
 u'after',
 u'above',
 u'below',
 u'to',
 u'from',
 u'up',
 u'down',
 u'in',
 u'out',
 u'on',
 u'off',
 u'over',
 u'under',
 u'again',
 u'further',
 u'th